In [2]:
from sklearn.cluster import KMeans
import pandas as pd
import pickle
import string
import re
import string
from sklearn.model_selection import cross_validate as cross_validation, ShuffleSplit, cross_val_score, train_test_split, KFold
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import classification_report, accuracy_score, auc
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder
import pandas.util.testing as tm

C:\Users\Asus\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=True, eps=np.finfo(np.float).eps,
C:\Users\Asus\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:167: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=True, e

In [58]:
data2 = pd.read_pickle("Cyclical_encoded_data.pkl")

In [60]:
print(data2.columns.tolist())
data2 = data2.drop(['ID_HH', 'RHoMIS_ID'], axis=1)
data2

['YEAR', 'ITERATION', 'ID_HH', 'RHoMIS_ID', 'GPS_LAT', 'GPS_LON', 'GPS_ALT', 'HHsizemembers', 'HHsizeMAE', 'LandOwned', 'LandCultivated', 'LivestockHoldings', 'NrofMonthsFoodInsecure', 'PPI_Threshold', 'PPI_Likelihood', 'score_HDDS_GoodSeason', 'score_HDDS_farmbasedGoodSeason', 'score_HDDS_purchasedGoodSeason', 'score_HDDS_BadSeason', 'score_HDDS_farmbasedBadSeason', 'score_HDDS_purchasedBadSeason', 'TVA_USD_PPP_pmae_pday', 'currency_conversion_factor', 'total_income_USD_PPP_pHH_Yr', 'offfarm_income_USD_PPP_pHH_Yr', 'farm_income_USD_PPP_pHH_Yr', 'value_farm_produce_USD_PPP_pHH_Yr', 'crop_sales_USD_PPP_pHH_Yr', 'value_crop_produce_USD_PPP_pHH_Yr', 'value_crop_consumed_USD_PPP_pHH_Yr', 'livestock_prodsales_USD_PPP_pHH_Yr', 'value_livestock_production_USD_PPP_pHH_Yr', 'value_livestock_prod_consumed_USD_PPP_pHH_Yr', 'Market_Orientation', 'Livestock_Orientation', 'Food_Availability_kCal_MAE_day', 'Food_Self_Sufficiency_kCal_MAE_day', 'NrofMonthsWildFoodCons', 'GHGEmissions', 'Gender_MaleCon

,YEAR,ITERATION,GPS_LAT,GPS_LON,GPS_ALT,HHsizemembers,HHsizeMAE,LandOwned,LandCultivated,LivestockHoldings,...,Country_<Palestine>,Country_<Peru>,Country_<Rwanda>,Country_<Senegal>,Country_<Sierra Leone>,Country_<South Africa>,Country_<Tanzania>,Country_<Uganda>,Country_<Vietnam>,Country_<Zambia>
0,2019.0,1,11.19,-0.95,254.00,7.0,5.56,10.0,2.0,1.04,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2019.0,1,11.19,-0.95,346.00,10.0,8.02,NaN,1.2,4.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2019.0,1,11.19,-0.99,324.00,6.0,4.69,2.0,0.8,0.30,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2019.0,1,11.19,-0.99,326.00,6.0,4.46,NaN,0.8,5.43,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2019.0,1,11.22,-1.02,308.00,8.0,6.52,10.0,4.0,2.12,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35723,2017.0,1,-13.82,32.67,1119.60,6.0,4.71,8.0,8.0,3.58,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
35724,2017.0,1,-13.82,32.67,1128.85,11.0,9.06,NaN,14.0,8.80,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
35725,2017.0,1,-13.82,32.67,1132.41,10.0,7.54,5.0,6.0,5.48,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
35726,2017.0,1,-13.82,32.67,1136.24,6.0,4.96,4.0,4.0,0.70,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [24]:
data = data2[['GPS_LAT', 'GPS_ALT', 'LandOwned', 'LandCultivated', 'LivestockHoldings', 'Food_Availability_kCal_MAE_day', 'Food_Self_Sufficiency_kCal_MAE_day',
'crop_sales_USD_PPP_pHH_Yr', 'value_crop_produce_USD_PPP_pHH_Yr', 'GHGEmissions']]

In [25]:
data

,GPS_LAT,GPS_ALT,LandOwned,LandCultivated,LivestockHoldings,Food_Availability_kCal_MAE_day,Food_Self_Sufficiency_kCal_MAE_day,crop_sales_USD_PPP_pHH_Yr,value_crop_produce_USD_PPP_pHH_Yr,GHGEmissions
0,11.19,254.00,10.0,2.0,1.04,4621.267353,754.360895,164.857702,242.318219,702.949550
1,11.19,346.00,NaN,1.2,4.00,2503.023263,2303.522017,0.000000,279.844923,94415.087935
2,11.19,324.00,2.0,0.8,0.30,381.108158,381.108158,0.000000,23.278518,113.654565
3,11.19,326.00,NaN,0.8,5.43,9948.141738,620.787518,228.969031,255.682085,43062.200002
4,11.22,308.00,10.0,4.0,2.12,2209.261281,1767.543491,22.896903,184.734710,13331.792068
...,...,...,...,...,...,...,...,...,...,...
35723,-13.82,1119.60,8.0,8.0,3.58,21491.305517,12770.090428,2970.040440,9187.374358,34884.136689
35724,-13.82,1128.85,NaN,14.0,8.80,11260.499490,4217.809731,4501.379943,8866.040295,79114.978038
35725,-13.82,1132.41,5.0,6.0,5.48,4247.172555,1221.939610,1083.897904,2007.669471,46350.365405
35726,-13.82,1136.24,4.0,4.0,0.70,5918.719620,5918.719620,0.000000,2026.134892,7452.589981


In [26]:
#mean for each feature of interest
variable_mean = {}
for i in data2.columns.tolist():
        variable_mean[i]= data2[i].mean()
print(variable_mean)

TypeError: Could not convert BF_2019_ADN_1_1BF_2019_ADN_2_1BF_2019_ADN_3_1BF_2019_ADN_4_1BF_2019_ADN_5_1BF_2019_ADN_6_1BF_2019_ADN_7_1BF_2019_ADN_8_1BF_2019_ADN_9_1BF_2019_ADN_10_1BF_2019_ADN_11_1BF_2019_ADN_12_1BF_2019_ADN_13_1BF_2019_ADN_14_1BF_2019_ADN_15_1BF_2019_ADN_16_1BF_2019_ADN_17_1BF_2019_ADN_18_1BF_2019_ADN_19_1BF_2019_ADN_20_1BF_2019_ADN_21_1BF_2019_ADN_22_1BF_2019_ADN_23_1BF_2019_ADN_24_1BF_2019_ADN_25_1BF_2019_ADN_26_1BF_2019_ADN_27_1BF_2019_ADN_28_1BF_2019_ADN_29_1BF_2019_ADN_30_1BF_2019_ADN_31_1BF_2019_ADN_32_1BF_2019_ADN_33_1BF_2019_ADN_34_1BF_2019_ADN_35_1BF_2019_ADN_36_1BF_2019_ADN_37_1BF_2019_ADN_38_1BF_2019_ADN_39_1BF_2019_ADN_40_1BF_2019_ADN_41_1BF_2019_ADN_42_1BF_2019_ADN_43_1BF_2019_ADN_44_1BF_2019_ADN_45_1BF_2019_ADN_46_1BF_2019_ADN_47_1BF_2019_ADN_48_1BF_2019_ADN_49_1BF_2019_ADN_50_1BF_2019_ADN_51_1BF_2019_ADN_52_1BF_2019_ADN_53_1BF_2019_ADN_54_1BF_2019_ADN_55_1BF_2019_ADN_56_1BF_2019_ADN_57_1BF_2019_ADN_58_1BF_2019_ADN_59_1BF_2019_ADN_60_1BF_2019_ADN_61_1BF_2

In [27]:
#mean for each feature of interest 
data2.mean()

YEAR                      2018.181576
ITERATION                    1.028859
GPS_LAT                     11.723959
GPS_LON                     23.046392
GPS_ALT                    940.749333
                             ...     
Country_<South Africa>       0.003695
Country_<Tanzania>           0.080614
Country_<Uganda>             0.033281
Country_<Vietnam>            0.043470
Country_<Zambia>             0.033393
Length: 98, dtype: float64

In [28]:
#replace nan values with the appropriate mean
data_no_nan = data2.fillna(data2.mean())

In [29]:
data_no_nan

,YEAR,ITERATION,ID_HH,RHoMIS_ID,GPS_LAT,GPS_LON,GPS_ALT,HHsizemembers,HHsizeMAE,LandOwned,...,Country_<Palestine>,Country_<Peru>,Country_<Rwanda>,Country_<Senegal>,Country_<Sierra Leone>,Country_<South Africa>,Country_<Tanzania>,Country_<Uganda>,Country_<Vietnam>,Country_<Zambia>
0,2019.0,1,BF_2019_ADN_1_1,BF_2019_ADN_1_1,11.19,-0.95,254.00,7.0,5.56,10.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2019.0,1,BF_2019_ADN_2_1,BF_2019_ADN_2_1,11.19,-0.95,346.00,10.0,8.02,13.159772,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2019.0,1,BF_2019_ADN_3_1,BF_2019_ADN_3_1,11.19,-0.99,324.00,6.0,4.69,2.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2019.0,1,BF_2019_ADN_4_1,BF_2019_ADN_4_1,11.19,-0.99,326.00,6.0,4.46,13.159772,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2019.0,1,BF_2019_ADN_5_1,BF_2019_ADN_5_1,11.22,-1.02,308.00,8.0,6.52,10.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35723,2017.0,1,ZM_2017_SCN_606_1,ZM_2017_SCN_606_1,-13.82,32.67,1119.60,6.0,4.71,8.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
35724,2017.0,1,ZM_2017_SCN_607_1,ZM_2017_SCN_607_1,-13.82,32.67,1128.85,11.0,9.06,13.159772,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
35725,2017.0,1,ZM_2017_SCN_608_1,ZM_2017_SCN_608_1,-13.82,32.67,1132.41,10.0,7.54,5.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
35726,2017.0,1,ZM_2017_SCN_609_1,ZM_2017_SCN_609_1,-13.82,32.67,1136.24,6.0,4.96,4.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [30]:
#check if there's still any nan values
data_no_nan.isnull().values.any()

False

In [67]:
from sklearn.preprocessing import Imputer
imputer = Imputer(missing_values='NaN', strategy='mean', axis=1)
imputer = imputer.fit(data2)
imputed_data = imputer.transform(data2.values)
imputed_data


C:\Users\Asus\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:66: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


array([[ 2.019e+03,  1.000e+00,  1.119e+01, ...,  0.000e+00,  0.000e+00,
         0.000e+00],
       [ 2.019e+03,  1.000e+00,  1.119e+01, ...,  0.000e+00,  0.000e+00,
         0.000e+00],
       [ 2.019e+03,  1.000e+00,  1.119e+01, ...,  0.000e+00,  0.000e+00,
         0.000e+00],
       ...,
       [ 2.017e+03,  1.000e+00, -1.382e+01, ...,  0.000e+00,  0.000e+00,
         1.000e+00],
       [ 2.017e+03,  1.000e+00, -1.382e+01, ...,  0.000e+00,  0.000e+00,
         1.000e+00],
       [ 2.017e+03,  1.000e+00, -1.382e+01, ...,  0.000e+00,  0.000e+00,
         1.000e+00]])

In [69]:
imputed_data = pd.DataFrame(imputed_data)
imputed_data
imputed_data.columns =[data2.columns.tolist()]

In [64]:
imputed_data
imputed_data.columns =[data2.columns.tolist()]

In [66]:
imputed_data.to_pickle('./mean_imputed.pkl')

In [70]:
imputed_data

,YEAR,ITERATION,GPS_LAT,GPS_LON,GPS_ALT,HHsizemembers,HHsizeMAE,LandOwned,LandCultivated,LivestockHoldings,...,Country_<Palestine>,Country_<Peru>,Country_<Rwanda>,Country_<Senegal>,Country_<Sierra Leone>,Country_<South Africa>,Country_<Tanzania>,Country_<Uganda>,Country_<Vietnam>,Country_<Zambia>
0,2019.0,1.0,11.19,-0.95,254.00,7.0,5.56,10.000000,2.0,1.04,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2019.0,1.0,11.19,-0.95,346.00,10.0,8.02,1083.047816,1.2,4.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2019.0,1.0,11.19,-0.99,324.00,6.0,4.69,2.000000,0.8,0.30,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2019.0,1.0,11.19,-0.99,326.00,6.0,4.46,612.007008,0.8,5.43,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2019.0,1.0,11.22,-1.02,308.00,8.0,6.52,10.000000,4.0,2.12,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35721,2017.0,1.0,-13.82,32.67,1119.60,6.0,4.71,8.000000,8.0,3.58,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
35722,2017.0,1.0,-13.82,32.67,1128.85,11.0,9.06,1389.539030,14.0,8.80,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
35723,2017.0,1.0,-13.82,32.67,1132.41,10.0,7.54,5.000000,6.0,5.48,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
35724,2017.0,1.0,-13.82,32.67,1136.24,6.0,4.96,4.000000,4.0,0.70,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
